In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [21]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [22]:
print("Training data shape = {}".format(train_data.shape))
print("Test data shape = {}".format(test_data.shape))
train_data.head()

Training data shape = (891, 12)
Test data shape = (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Feature Engineering

In [23]:
train_data = train_data.drop(columns=['Name','Cabin'])
train_data['family_member'] = train_data['SibSp'] + train_data['Parch']
train_data = train_data.drop(columns=['SibSp', 'Parch'])

In [24]:
test_data = test_data.drop(columns=['Name','Cabin'])
test_data['family_member'] = test_data['SibSp'] + test_data['Parch']
test_data = test_data.drop(columns=['SibSp', 'Parch'])

In [25]:
train_data.fillna(0,inplace=True)
test_data.fillna(0,inplace=True)

In [26]:
X = train_data.drop(columns=['Survived'])
Y = train_data['Survived']

cate_features_index = np.where(X.dtypes != float)[0]

# Modelling With CatBoost

In [27]:
from catboost import CatBoostClassifier, cv, Pool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [28]:
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,train_size=0.90,random_state=42)
clf =CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42,learning_rate=0.05)

In [33]:
clf.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest), early_stopping_rounds=50,verbose=0)

0:	learn: 0.7952559	test: 0.8111111	best: 0.8111111 (0)	total: 4.37ms	remaining: 4.36s
1:	learn: 0.7965044	test: 0.7888889	best: 0.8111111 (0)	total: 6.46ms	remaining: 3.22s
2:	learn: 0.7965044	test: 0.7888889	best: 0.8111111 (0)	total: 9.88ms	remaining: 3.28s
3:	learn: 0.7990012	test: 0.8222222	best: 0.8222222 (3)	total: 13.2ms	remaining: 3.28s
4:	learn: 0.7990012	test: 0.8222222	best: 0.8222222 (3)	total: 14.6ms	remaining: 2.9s
5:	learn: 0.8139825	test: 0.7666667	best: 0.8222222 (3)	total: 17.8ms	remaining: 2.94s
6:	learn: 0.8127341	test: 0.8222222	best: 0.8222222 (3)	total: 21.1ms	remaining: 3s
7:	learn: 0.8127341	test: 0.8222222	best: 0.8222222 (3)	total: 22.5ms	remaining: 2.79s
8:	learn: 0.8164794	test: 0.8222222	best: 0.8222222 (3)	total: 23.5ms	remaining: 2.58s
9:	learn: 0.8164794	test: 0.8222222	best: 0.8222222 (3)	total: 25.7ms	remaining: 2.55s
10:	learn: 0.8139825	test: 0.8222222	best: 0.8222222 (3)	total: 28.6ms	remaining: 2.57s
11:	learn: 0.8139825	test: 0.8222222	best: 0.8

In [30]:
test_id = test_data.PassengerId
test_data.isnull().sum()

PassengerId      0
Pclass           0
Sex              0
Age              0
Ticket           0
Fare             0
Embarked         0
family_member    0
dtype: int64

In [31]:
prediction = clf.predict(test_data)

# Saving Output

In [32]:
df_sub = pd.DataFrame()
df_sub['PassengerId'] = test_id
df_sub['Survived'] = prediction.astype(np.int)

df_sub.to_csv('gender_submission.csv', index=False)